In [15]:
import pandas as pd
import wget
import time
import os
import zipfile
import numpy as np
import shutil

from argparse import ArgumentParser

from  tqdm import tqdm
import sys
sys.path.insert(0, '../../')
from utils.utils_date import *

In [18]:
def download_file(url, filename_path):
    
    
    directory = os.path.dirname(filename_path)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    start_time = time.time()
    filename = wget.download(url,filename_path)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return filename

In [156]:
def main(path_to_save, full_path_date, full_path_save):

    print('Load data from open data paris API and remove them')


    url_data = 'https://opendata.stif.info/explore/dataset/histo-validations/files/ff00c4d0d1ac37b823f5ba854a895d8e/download/'
    filename_data = download_file(url_data, path_to_save + 'data_2017S1.zip')
    zip_ref = zipfile.ZipFile(filename_data, 'r')
    zip_ref.extractall(os.path.dirname(filename_data) + '/')
    zip_ref.close()

    print(filename_data)
    os.remove(filename_data)
    os.remove(path_to_save + 'DATA PUBLIEES S1 2017/2017S1_NB_SURFACE.txt')
    os.remove(path_to_save + 'DATA PUBLIEES S1 2017/2017S1_PROFIL_SURFACE.txt')

    filename_profil = path_to_save + 'DATA PUBLIEES S1 2017/2017S1_PROFIL_FER.txt'
    filename_validations = path_to_save + 'DATA PUBLIEES S1 2017/2017S1_NB_FER.txt'

    df_validations = pd.read_csv(filename_validations, sep='\t')
    df_profil = pd.read_csv(filename_profil, sep='\t')

    shutil.rmtree(path_to_save + 'DATA PUBLIEES S1 2017/', ignore_errors=True)

    url_data = 'https://opendata.stif.info/explore/dataset/emplacement-des-gares-idf/download/?format=csv&timezone=America/New_York&use_labels_for_header=true'
    filename_data = download_file(url_data, path_to_save + 'station_information.csv')

    df_station = pd.read_csv(filename_data, sep=';')

    print(filename_data)
    os.remove(filename_data)


    print('Clean data')


    dict_idrl_csa = dict(df_validations[['ID_REFA_LDA', 'CODE_STIF_ARRET']].drop_duplicates().values)
    dict_csa_idrl = dict(df_validations[['CODE_STIF_ARRET', 'ID_REFA_LDA']].drop_duplicates().values)

    # Modif df_validations

    df_validations = df_validations[['JOUR', 'ID_REFA_LDA', 'NB_VALD']]
    df_validations = df_validations.dropna()
    df_validations['NB_VALD'] = np.array([1 if i[0] == 'M' else i for i in df_validations['NB_VALD'].values]).astype(
        int)
    df_validations = df_validations[df_validations.ID_REFA_LDA != '?']
    df_validations.ID_REFA_LDA = df_validations.ID_REFA_LDA.values.astype(int).astype(str)
    df_validations['JOUR'] = [i.split('/')[2] + '-' + i.split('/')[1] + '-' + i.split('/')[0] for i in
                              df_validations['JOUR']]

    # Modif df_profil
    df_profil = df_profil[['TRNC_HORR_60', 'ID_REFA_LDA', 'CAT_JOUR', 'pourc_validations']]
    df_profil = df_profil.dropna()
    df_profil = df_profil[df_profil.TRNC_HORR_60 != 'ND']
    df_profil = df_profil[df_profil.ID_REFA_LDA != '?']
    df_profil.ID_REFA_LDA = df_profil.ID_REFA_LDA.values.astype(int).astype(str)
    df_profil['TRNC_HORR_60'] = df_profil['TRNC_HORR_60'].values.astype(str)
    df_profil = df_profil.drop_duplicates()

    # Modif df_station
    df_station['GARES_ID'] = df_station['GARES_ID'].values.astype(str)


    print('Create dictionaries')


    # 1 - dict_idstation_typeofday_percentage with df_profil
    trnc_horr_60 = ['0H-1H', '1H-2H', '2H-3H', '3H-4H', '4H-5H', '5H-6H', '6H-7H', '7H-8H', '8H-9H', '9H-10H',
                    '10H-11H', '11H-12H', '12H-13H', '13H-14H', '14H-15H', '15H-16H', '16H-17H', '17H-18H',
                    '18H-19H', '19H-20H', '20H-21H', '21H-22H', '22H-23H', '23H-0H']

    time_60 = [i[11:] for i in
               build_timestamp_list("2000-01-01 00:00:00", "2000-01-01 23:00:00", time_step_second=60 * 60)]

    df = pd.pivot_table(df_profil, values=['pourc_validations'],
                        index=['ID_REFA_LDA', 'CAT_JOUR'],
                        columns='TRNC_HORR_60', fill_value=0)

    df.columns = df.columns.droplevel(0)
    df.columns.name = None
    df = df.reset_index()
    df = df[['ID_REFA_LDA', 'CAT_JOUR'] + trnc_horr_60]

    list_index_incomplete = df['ID_REFA_LDA'].drop_duplicates().values[
        df[['ID_REFA_LDA']].groupby('ID_REFA_LDA').size().values != 5]

    for i in list_index_incomplete:
        df = df[df['ID_REFA_LDA'] != i]

    dict_idstation_typeofday_percentage = dict([((i[0], i[1]), np.array(i[2:]).astype(float)) for i in df.values])

    # 2 - dict_idstation_day_sumvalidation with df_profil

    df = df_validations.groupby(['JOUR', 'ID_REFA_LDA']).sum().reset_index()
    df = df[['ID_REFA_LDA', 'JOUR', 'NB_VALD']]
    dict_idstation_day_sumvalidation = dict([((i[0], i[1]), i[2]) for i in df.values])

    # 3 - dict_day_category1
    df_exogenous = pd.read_csv(full_path_date)
    dict_day_category1 = dict([(i[:10], j) for i, j in df_exogenous[['Datetime', 'Category1']].values])


    print('Create and reshape dataframe datetime_values')


    id_stations_validations = np.array(sorted(set([i[0] for i in dict_idstation_day_sumvalidation.keys()])))
    id_stations_profil = np.array(sorted(set([i[0] for i in dict_idstation_typeofday_percentage.keys()])))
    id_stations = list(filter(lambda x: x in id_stations_validations, id_stations_profil))
    day_list = sorted(df_validations['JOUR'].drop_duplicates().tolist())

    dict_trnc_time = dict(zip(trnc_horr_60, time_60))

    values = [build_timestamp_list(day + " 00:00:00", day + " 23:00:00", time_step_second=60 * 60)
              for day in day_list]
    values = [item for sublist in values for item in sublist]
    df_datetime_values = pd.DataFrame(data=values, columns=['Datetime'])

    columns = ['Date'] + trnc_horr_60
    list_df = []
    value_incomplete = 0

    for ids in tqdm(id_stations):
        data = []
        for day in day_list:
            typeofday = dict_day_category1[day]
            try:
                v = dict_idstation_day_sumvalidation[(ids, day)] * dict_idstation_typeofday_percentage[(ids, typeofday)]
            except:
                value_incomplete = +1
                v = np.zeros(24)
            data.append([day] + list(v))

        df = pd.DataFrame(data=data, columns=columns)

        df = df.set_index("Date").stack().reset_index()
        df.columns = ['Date', 'trnc_horr_60', ids]
        df['time'] = [dict_trnc_time[i] for i in df['trnc_horr_60'].values]
        df['Datetime'] = [i + " " + j for i, j in df[['Date', 'time']].values]
        list_df.append(df[['Datetime', ids]].set_index('Datetime'))

    print("Value incomplete {:.4f}%".format((value_incomplete / (len(id_stations) * len(day_list) * 1.)) * 100))

    df_datetime_values = df_datetime_values.set_index('Datetime').join(list_df).reset_index()


    print('Filter on stations of Metro line 1 and 7 for the test')


    df_station_ = df_station[['MODE_', 'LIGNE', 'GARES_ID']].groupby(['MODE_', 'LIGNE'])['GARES_ID'].apply(
        lambda x: "%s" % ','.join(x)).reset_index()
    dict_modeligne_csa = dict(
        [((i[0], i[1]), i[2].split(',')) for i in df_station_[['MODE_', 'LIGNE', 'GARES_ID']].values])

    filter_modeligne = [('Metro', '1'), ('Metro', '7')]
    filter_station_id = []
    for i in filter_modeligne:

        for j in dict_modeligne_csa[i]:

            try:
                filter_station_id.append(dict_csa_idrl[j])
            except:
                print("Fail to link station with CodeStifArret {} with IdRefaLda".format(j))

    filter_station_id_ = []
    for i in filter_station_id:
        if i in id_stations:
            filter_station_id_.append(i)
        else:
            print("Fail to link station id {} with datetime_values file".format(i))

    df_datetime_values_filtered = df_datetime_values[['Datetime'] + filter_station_id_]
    df_datetime_values_filtered.to_csv(full_path_save, index=False)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("-p", "--path_to_save", default="../../data/")
    parser.add_argument("-fpd", "--full_path_date", default="../../data/date_file_2013_2018_included_test.csv")
    parser.add_argument("-fps", "--full_path_save",
                        default="../../data/observation_file_2017-01-01_2017-06-30_included_test.csv")

    args = parser.parse_args()
    path_to_save = args.path_to_save
    full_path_date = args.full_path_date
    full_path_save = args.full_path_save

    main(path_to_save, full_path_date, full_path_save)



usage: ipykernel_launcher.py [-h] [-p PATH_TO_SAVE] [-fpd FULL_PATH_DATE]
                             [-fps FULL_PATH_SAVE]
ipykernel_launcher.py: error: ambiguous option: -f could match -fps, -fpd


SystemExit: 2

/opt/conda/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pandas as pd

In [6]:
df_data = pd.read_csv('../../data/observation_file_2017-01-01_2017-06-30_included_test.csv')

In [11]:
df_data

,Datetime,71634,71650,71442,71654,71743,71328,71305,71517,71284,...,71201,72460,70488,71076,70604,73695,70143,70248,71001,73615
0,2017-01-01 00:00:00,2245.93,780.70,1075.36,2288.44,14696.44,12420.87,2417.80,1.094460e+04,1066.72,...,1021.76,3901.45,732.99,2213.78,903.21,1179.78,1470.42,483.00,1564.62,384.02
1,2017-01-01 01:00:00,188.91,147.70,206.80,326.92,2179.32,965.58,241.78,1.233950e+03,90.40,...,248.00,329.70,138.30,202.67,106.26,84.27,175.05,115.50,246.12,63.30
2,2017-01-01 02:00:00,20.99,10.55,20.68,29.72,167.64,43.89,0.00,0.000000e+00,18.08,...,19.84,0.00,13.83,31.18,17.71,0.00,35.01,10.50,35.16,12.66
3,2017-01-01 03:00:00,0.00,0.00,0.00,0.00,55.88,0.00,0.00,0.000000e+00,9.04,...,4.96,0.00,13.83,15.59,0.00,0.00,35.01,0.00,0.00,0.00
4,2017-01-01 04:00:00,83.96,0.00,0.00,0.00,0.00,43.89,51.81,2.146000e+03,9.04,...,4.96,54.95,55.32,0.00,0.00,28.09,35.01,0.00,0.00,0.00
5,2017-01-01 05:00:00,1448.31,464.20,1075.36,1723.76,8046.72,3160.08,1519.76,7.940200e+03,325.44,...,272.80,6648.95,1714.92,685.96,2337.72,4297.77,4726.35,966.00,773.52,409.34
6,2017-01-01 06:00:00,2497.81,1033.90,1778.48,3417.80,6370.32,4125.66,1986.05,2.135270e+04,361.60,...,322.40,12583.55,3056.43,1371.92,4321.24,7500.03,9172.62,2005.50,1652.52,637.22
7,2017-01-01 07:00:00,3988.10,1899.00,4115.32,5052.40,5196.84,4564.56,1433.41,3.610645e+04,569.52,...,580.32,20001.80,4342.62,2759.43,5932.85,10758.47,12253.50,3066.00,2795.22,1367.28
8,2017-01-01 08:00:00,5877.20,3597.55,6555.56,8589.08,7990.84,6364.05,1157.09,4.946530e+04,1003.44,...,1061.44,26485.90,6458.61,4287.25,7562.17,14578.71,17259.93,4704.00,5344.32,1848.36
9,2017-01-01 09:00:00,6968.68,5180.05,9616.20,11323.32,11678.92,6671.28,1364.33,5.719090e+04,1130.00,...,1334.24,33684.35,8961.84,6017.74,10413.48,17809.06,21146.04,6489.00,7524.24,2270.36


In [24]:
path_to_save = '../../data/'
url_data = 'https://opendata.stif.info/explore/dataset/emplacement-des-gares-idf/download/?format=csv&timezone=America/New_York&use_labels_for_header=true'
filename_data = download_file(url_data, path_to_save+'station_information.csv')

df_station = pd.read_csv(filename_data, sep=';')

os.remove(filename_data)

--- 1.5678651332855225 seconds ---


In [25]:
url_data = 'https://opendata.stif.info/explore/dataset/histo-validations/files/ff00c4d0d1ac37b823f5ba854a895d8e/download/'
filename_data = download_file(url_data, path_to_save + 'data_2017S1.zip')
zip_ref = zipfile.ZipFile(filename_data, 'r')
zip_ref.extractall(os.path.dirname(filename_data) + '/')
zip_ref.close()

print(filename_data)
os.remove(filename_data)
os.remove(path_to_save + 'DATA PUBLIEES S1 2017/2017S1_NB_SURFACE.txt')
os.remove(path_to_save + 'DATA PUBLIEES S1 2017/2017S1_PROFIL_SURFACE.txt')

filename_profil = path_to_save + 'DATA PUBLIEES S1 2017/2017S1_PROFIL_FER.txt'
filename_validations = path_to_save + 'DATA PUBLIEES S1 2017/2017S1_NB_FER.txt'

df_validations = pd.read_csv(filename_validations, sep='\t')
df_profil = pd.read_csv(filename_profil, sep='\t')

shutil.rmtree(path_to_save + 'DATA PUBLIEES S1 2017/', ignore_errors=True)

--- 7.346240282058716 seconds ---
../../data/data_2017S1.zip


In [47]:
df_profil

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CAT_JOUR,TRNC_HORR_60,pourc_validations
0,100,110,1,PORTE MAILLOT,71379,DIJFP,0H-1H,2.10
1,100,110,1,PORTE MAILLOT,71379,DIJFP,10H-11H,3.92
2,100,110,1,PORTE MAILLOT,71379,DIJFP,11H-12H,4.37
3,100,110,1,PORTE MAILLOT,71379,DIJFP,12H-13H,4.94
4,100,110,1,PORTE MAILLOT,71379,DIJFP,13H-14H,5.54
5,100,110,1,PORTE MAILLOT,71379,DIJFP,14H-15H,6.35
6,100,110,1,PORTE MAILLOT,71379,DIJFP,15H-16H,7.69
7,100,110,1,PORTE MAILLOT,71379,DIJFP,16H-17H,7.93
8,100,110,1,PORTE MAILLOT,71379,DIJFP,17H-18H,8.38
9,100,110,1,PORTE MAILLOT,71379,DIJFP,18H-19H,9.17


In [46]:
df_station.columns


Index(['Geo Point', 'Geo Shape', 'ID_REF_ZDL', 'ID_REF_ZDE', 'GARES_ID',
       'NOM_GARE', 'NOMLONG', 'NOM_IV', 'NUM_MOD', 'MODE_', 'FER', 'TRAIN',
       'RER', 'METRO', 'TRAMWAY', 'NAVETTE', 'VAL', 'TERFER', 'TERTRAIN',
       'TERRER', 'TERMETRO', 'TERTRAM', 'TERNAVETTE', 'TERVAL', 'IDREFLIGA',
       'IDREFLIGC', 'LIGNE', 'COD_LIGF', 'LIGNE_CODE', 'INDICE_LIG', 'RESEAU',
       'RES_COM', 'COD_RESF', 'RES_STIF', 'EXPLOITANT', 'NUM_PSR', 'IDF',
       'PRINCIPAL', 'X', 'Y', 'EDITEUR', 'EDIT_DATE', 'TERMINUS'],
      dtype='object')